# Homework #8
### Name: Revan Minnam


#### Define the location of the data sets and create the text RDDs

In [1]:
path = '/public/bmort/spark/'
storeRDD = sc.textFile(path + 'wegmans_store_master.txt')
itemRDD = sc.textFile(path + 'wegmans_item_master.txt')
customerRDD = sc.textFile(path + 'wegmans_customer_master.txt')
postransRDD = sc.textFile(path + 'partial_transaction.dat')

#### Define a pipe-delimited parsing function to split data based on the pipe character and repackage the returned results as a Row object.

In [ ]:
from datetime import datetime
from pyspark.sql import Row
def parseStore(s):
    l = s.split('|')
    return Row(store_num = int(l[0]), 
               store_name = l[1],               
               store_zone = l[2],
               store_city = l[3], 
               store_state = l[4], 
               store_type = int(l[5]))
def parseItem(s):
    l = s.split('|')
    return Row(item_number = int(l[0]), 
               dept_categ_class = l[1],               
               item_des = l[2],
               item_unt_qty = float(l[3]), 
               size_unit_desc = l[4], 
               brand_code = l[5], 
               dept_num = int(l[6]), 
               dept_name = l[7], 
               categ_num = int(l[8]), 
               categ_name = l[9], 
               class_num = int(l[10]),
               class_name = l[11])
def parseCustomer(s):
    l = s.split('|')
    return Row(hshld_acct = int(l[0]),
               birth_yr_head_hh = l[1],
               hh_income = l[2],
               hh_size = l[3],
               adult_count = l[4],
               child_count = l[5],
               birth_yr_oldest = l[6],
               birth_yr_youngest = l[7],
               bad_address = l[8],
               privacy = l[9],
               application_date = datetime.strptime(l[10],'%Y-%m-%d'),
               wine_email_sent = int(l[11]),
               wine_email_open = int(l[12]),
               wine_email_click = int(l[13]))
def parsePostrans(s):
    l = s.split('|')
    return Row(hshld_acct = int(l[0]),
               acct_num = int(l[1]),
               trans_num = int(l[2]),
               trans_date = datetime.strptime(l[3],'%Y-%m-%d'),
               store_num = int(l[4]),
               item_number = int(l[5]),
               dept_categ_class = l[6],
               unit_count = int(l[7]),
               net_sales = float(l[8]),
               gross_sales = float(l[9]),
               manuf_coupon = float(l[10]))


#### Generate the row-based RDDs

In [ ]:
storeRowRDD = storeRDD.map(parseStore)
itemRowRDD = itemRDD.map(parseItem)
customerRowRDD = customerRDD.map(parseCustomer)
postransRowRDD = postransRDD.map(parsePostrans)

#### Generate data frames from the RDDs

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
storeDF = sqlContext.createDataFrame(storeRowRDD)
itemDF = sqlContext.createDataFrame(itemRowRDD)
customerDF = sqlContext.createDataFrame(customerRowRDD)
postransDF = sqlContext.createDataFrame(postransRowRDD)

#### Create temporary views for the data frames for use in a SQL context

In [ ]:
storeDF.createOrReplaceTempView('store')
itemDF.createOrReplaceTempView('item')
customerDF.createOrReplaceTempView('customer')
postransDF.createOrReplaceTempView('postrans')

## Question 1
* In the lecture we determined the total sales for each store. Now determine the total sales for each region. Your final output should have 2 columns (store_zone and total_sales) and 5 rows, corresponding to the 5 regions. Hint: Begin by joining the point of sale trasaction data frame with the store data frame. (It may take a few minutes to run your code.)

In [ ]:
# Complete code here to produce the output
from pyspark.sql import functions as F
postransDF.join(storeDF, 'store_num') \
    .groupBy('store_zone') \
    .agg(F.sum('gross_sales').alias('total_sales')) \
    .select('store_zone','total_sales') \
    .sort('total_sales', ascending=False) \
    .show()

## Question 2
* Calculate the total number of transactions (in the postransDF) for each store. Display the results in descending order by the number of transactions for the top 5 stores. Your final output should have 2 columns (store_name and count). The number of rows should be equal to 5. (It may take a few minutes to run your code.)

In [ ]:
# Complete code here to produce the output
postransDF.groupBy('store_num') \
    .agg(F.count('trans_num').alias('count')) \
    .join(storeDF, 'store_num') \
    .select('store_name', 'count') \
    .sort('count', ascending=False) \
    .show(5)